# Imports

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt


# ---
import sys; sys.path.append('../')

from commons.imgs_mean_std import *

from commons.dataset import *
from commons.imageutils import *

from modules.img_normalizer import *
from modules.train_functions import *

from modules.Dataset import *
from modules.EarlyStopper import *
from modules.Model import *

I0000 00:00:1735440722.881936  459791 gl_context.cc:357] GL version: 2.1 (2.1 INTEL-23.0.22), renderer: Intel(R) Iris(TM) Plus Graphics 645
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


# Constants & hyperparams

In [2]:
RANDOM_STATE=None

IMAGE_SIZE=220

BATCH_SIZE=32
NUM_EPOCHS=100

LEARNING_RATE=.001
MOMENTUM=.9

EARLY_PATIENCE=10
SCHEDULER_PATIENCE=6

W0000 00:00:1735440722.887183  459901 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


# Loading datasets

In [3]:
train_dataset = pd.read_csv(os.path.join(DATASETS_PATHS.norm_faces.info, 'train_dataset.csv'))
val_dataset = pd.read_csv(os.path.join(DATASETS_PATHS.norm_faces.info, 'val_dataset.csv'))
test_dataset = pd.read_csv(os.path.join(DATASETS_PATHS.norm_faces.info, 'test_dataset.csv'))

W0000 00:00:1735440722.897426  459897 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


# Creating custom datasets

In [ ]:
normalization= normalize(IMAGE_SIZE, MEAN, STD)

train_dataset = CD_TrippletsCreator(train_dataset, transform=normalization, data_augmentation_tranforms=[data_augmentation()])
val_dataset = CD_TrippletsCreator(val_dataset, transform=normalization)
test_dataset = CD_TrippletsCreator(test_dataset, transform=normalization)

# Dataloaders

In [5]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, pin_memory=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, pin_memory=True, drop_last=True)

# Early stopping

In [ ]:
def early_callback(**kwarks):
    print(" > Early Stop <")

early_stopper = EarlyStopper(EARLY_PATIENCE, .001, callback=early_callback)